In [2]:
import numpy as np
import matplotlib.pylab as plt
#from IPython import display
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.interpolate import splrep, BSpline
from ipywidgets import widgets, Layout, HTML, VBox, Label, Output
import qrcode
from IPython.display import Image
from scipy.integrate import odeint

In [1]:
# Create the title and description
title = widgets.HTML(value="<h1>STIR IT UP</h1>")

label_layout = widgets.Layout(width='600px', height='flex', align_items='flex-end')

text = """
At the quantum level, atoms can be found in various “states”, each belonging
to a specific energy. In our case, the atom has three of states, the first one
( |0> ) commonly known as the ground state, the first excited state ( |1> )
and its second excited state ( |2> ).<br><br>

Your job is simple, the atom starts at his ground state, however we need
to get it to its second excited state, and the only things you have under
your control are two laser pulses. Fortunately, you can change the shape of
the laser and fine-tune it to excite the atom. BEWARE! Quantum mechanics is
very different from our everyday experiences so don’t be astonished if you
incorrectly tuned your lasers to de-excite the atom or end up mixing some
of its states.<br><br>

The score will solely depend on how likely the atom is on the second excited
state at the final time. What is a good strategy here? Your everyday experience
may not help you much, you need to develop a “quantum way of thinking”.
"""

game_description_text = f"""
<div style="text-align: justify; font-size: 16px;"> 
    {text}
</div>
"""

# Create the images for the right side
image1 = widgets.Image(
    value=open('3states.png', 'rb').read(),
    format='png',
    width=500,
    height=400,
)

def update_laser_image():
    with image_output:
        clear_output(wait=True)
        image1 = widgets.Image(
                value=open('3states.png', 'rb').read(),
                format='png',
                width=500,
                height=400,
                )

image2 = widgets.Image(
    value=open('531px-Laser-symbol.svg.png', 'rb').read(),
    format='png',
    width=100,
    height=300,
)

game_description = widgets.HTML(value=game_description_text, layout=label_layout)

# Create the name input box
name_input = widgets.Text( 
    value='',
    placeholder='Enter your name',
    description='Name:',
    disabled=False
)

# Increase the font size of the input text box
name_input.style.font_size = '16px'  # Adjust the font size as needed

# Create the "Let's Go!" button
start_button = widgets.Button(
    description='Let\'s Go!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

# Increase the font size of the button text
start_button.style.font_size = '16px'  # Adjust the font size as needed

# Increase the width of the button
start_button.layout.width = '200px'  # Adjust the width as needed

bottom_left_box = widgets.HBox([image2, widgets.Label(), start_button, image2])

# Align the items in the HBox vertically in the center
bottom_left_box.layout.align_items = 'center'

# Create a VBox for the name input and button
name_input_box = widgets.VBox([name_input, bottom_left_box])

# Align the items in the VBox vertically in the center
name_input_box.layout.align_items = 'center'

# Create a VBox for the left side
left_text_box = widgets.VBox([game_description, name_input_box])

# Create an HBox for the right side
right_box = widgets.HBox([image1])

# Create a VBox for the entire content
main_box = widgets.VBox([title, widgets.HBox([left_text_box, right_box])])

# Center the main box within the display
# main_box.layout.align_items = 'center'

# Wrappe this main box as an output widget
main_box_output = widgets.Output()


# display(main_box)

# This will be the second page of the game after the introduction page

def Give_me_the_dynamics(user_name):

    global highScore

    currentScore = 0
        
    # Backend functions

    def Plot_Population(PSI_t, sim_t):
        with plot_output3: 
            clear_output(wait=True)
            fig, ax = plt.subplots()
            ax.plot(sim_t, np.abs(PSI_t[:, 0] + 1j*PSI_t[:, 3])**2, color = 'grey', linestyle = '--', label = 'State 1')
            ax.plot(sim_t, np.abs(PSI_t[:, 1] + 1j*PSI_t[:, 4])**2, color = 'grey', label = 'State 2')
            ax.plot(sim_t, np.abs(PSI_t[:, 2] + 1j*PSI_t[:, 5])**2, color = 'r', label = 'Score')
            ax.legend()
            ax.set_xlabel('time')
            ax.set_ylabel('Score')
            ax.set_title('State Populations [%]')
            plt.rcParams['lines.linewidth'] = 2
            plt.rcParams['axes.labelsize'] = 18
            plt.rcParams['axes.titlesize'] = 20
            plt.tick_params(axis='both', which='major', labelsize=18)
            plt.show()
            score_explanation = widgets.Label(str(currentScore) + '% of the population resulted at the 3rd state at the end of the run!')
            score_explanation.style.font_size = '16px'  
            currentScoreLabel = widgets.Label('Your Current Score: ' + str(currentScore))
            currentScoreLabel.style.font_size = '16px' 
            highScoreLabel = widgets.Label('Your High Score: ' + str(highScore))
            highScoreLabel.style.font_size = '16px' 
            score_layout = widgets.VBox([score_explanation, currentScoreLabel, highScoreLabel])
            display(score_layout)

    def Omega_Smooth(Omega_Inputs, T_end):
    # Pre-established Time-Domain
        T_axis = np.linspace(0, T_end, len(Omega_Inputs))

    # Random Values given by User
        tck = splrep(T_axis, Omega_Inputs, s=0)

    # Making the random values into Smooth Function (time below is also pre-established)
        t_smooth = np.linspace(0, T_end, 500)
        Pulse_smooth = BSpline(*tck)(t_smooth)

        return Pulse_smooth
    
    def dPsi_dt(Psi_t, t, params):
    
        # Unpack parameters
        D, Omega_P_array, Omega_S_array, TimePoints = params
    
        # Laser Pulses 
        #Omega_P = Omega_p(t)
        #Omega_S = Omega_s(t)
        Omega_P = np.interp(t, TimePoints, Omega_P_array)
        Omega_S = np.interp(t, TimePoints, Omega_S_array)
    
        # Real and Imaginary State Vectors
        S_R = Psi_t[:3]
        S_I = Psi_t[3:]

        # Defining the 1st Order Ode's
        dS_R_dt = np.array([ 0.5 * Omega_P * S_I[1],
                             0.5 * Omega_P * S_I[0] - D * S_I[1] + 0.5 * Omega_S * S_I[2],
                             0.5 * Omega_S * S_I[1]  ])

        dS_I_dt = np.array([ -0.5 * Omega_P * S_R[1],
                             -0.5 * Omega_P * S_R[0] + D * S_R[1] -  0.5 * Omega_S * S_R[2],
                             -0.5 * Omega_S * S_R[1] ])
    
        dS_dt = np.concatenate((dS_R_dt, dS_I_dt))
 
        return dS_dt

    ## Detuning Paramter
    D = -5.0
    
    ## Time grid 
    T_end = 10
    sim_t = np.linspace(0, T_end, 500)
    N_t = len(sim_t)
    
    # Getting Slider Values
    User_input_P = [slider.value for slider in point_sliders1]
    User_input_S = [slider.value for slider in point_sliders2]
    
    User_input_P.insert(0, 0)
    User_input_P.append(0)
    User_input_P.append(0)
    User_input_S.insert(0, 0)
    User_input_S.append(0)
    User_input_S.append(0)
    
    # We smooth the Values
    Omega_P1_Smooth = Omega_Smooth(User_input_P, T_end)
    Omega_S1_Smooth = Omega_Smooth(User_input_S, T_end)
    

    ## Set the initial condition (a vector of 6 elements, where the first 3 are real and the last 3 are imaginary)
    IC = np.zeros((6,))
    IC[0] = 1

    # Pack parameters for the ODE solver
    params = (D, Omega_P1_Smooth, Omega_S1_Smooth, sim_t)

    ## Solve the system using odeint
    PSI_t = odeint(dPsi_dt, IC, sim_t, args=(params,))
    SCORE = np.abs(PSI_t[-1, 2] + 1j*PSI_t[-1, 5])**2
    currentScore = round(100*SCORE , 2)

    if currentScore > int(highScore):
        highScore = currentScore


    Plot_Population(PSI_t, sim_t)

def generate_color_gradient_viridian_to_green(num_points):
    # Create a color map from viridis
    cmap = plt.get_cmap('viridis')

    # Create a list of evenly spaced values from 0 to 1
    values = np.linspace(0, 1, num_points)

    # Map the values to colors in the colormap
    colors = [cmap(value) for value in values]

    # Set the first and last colors to black
    colors[0] = 'black'
    colors[-1] = 'black'

    return colors

point_colors1 = generate_color_gradient_viridian_to_green(11)


def update_plot(point1_1, point2_1, point3_1, point4_1, point5_1, point6_1, point7_1, point8_1, point9_1,
                point1_2, point2_2, point3_2, point4_2, point5_2, point6_2, point7_2, point8_2, point9_2, point_colors1):
    X = np.arange(0, 11)
    Y1 = np.array([0, point1_1, point2_1, point3_1, point4_1, point5_1, point6_1, point7_1, point8_1, point9_1, 0])
    Y2 = np.array([0, point1_2, point2_2, point3_2, point4_2, point5_2, point6_2, point7_2, point8_2, point9_2, 0])
    
    

    
    with plot_output1:
        clear_output(wait=True)
        plt.figure(figsize=(7, 5))
        plt.scatter(X, Y1, c=point_colors1, marker='o', s=100)  # Assign colors to points
        plt.title('Laser One')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.ylim(-12.5, 12.5)
        Omega_P1_Smooth = Omega_Smooth(Y1, 10)
        t_smooth = np.linspace(0, 10, 500)
        plt.plot(t_smooth, Omega_P1_Smooth, color='b')
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['axes.labelsize'] = 18
        plt.rcParams['axes.titlesize'] = 20
        plt.tick_params(axis='both', which='major', labelsize=18)
        plt.tight_layout()
        plt.show()

    with plot_output2:
        clear_output(wait=True)
        plt.figure(figsize=(7, 5))
        plt.scatter(X, Y2, c=point_colors1, marker='o', s=100)  # Assign colors to points
        plt.title('Laser Two')
        plt.xlabel('Time')
        plt.ylabel('Amplitude')
        plt.ylim(-12.5, 12.5)
        Omega_P2_Smooth = Omega_Smooth(Y2, 10)
        t_smooth = np.linspace(0, 10, 500)
        plt.plot(t_smooth, Omega_P2_Smooth, color='r')
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['axes.labelsize'] = 18
        plt.rcParams['axes.titlesize'] = 20
        plt.tick_params(axis='both', which='major', labelsize=18)
        plt.tight_layout()
        plt.show()



def Omega_Smooth(Omega_Inputs, T_end):
    # Pre-established Time-Domain
    T_axis = np.linspace(0, T_end, len(Omega_Inputs))

    # Random Values given by User
    tck = splrep(T_axis, Omega_Inputs, s=0)

    # Making the random values into Smooth Function (time below is also pre-established)
    t_smooth = np.linspace(0, T_end, 500)
    Pulse_smooth = BSpline(*tck)(t_smooth)

    return Pulse_smooth

global user_name 
user_name = ''

# Initial values for the points
initial_points1 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
initial_points2 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

# Define a function to generate a color gradient from yellow to green
def generate_color_gradient(num_points):
    colors = []
    for i in range(num_points):
        # Interpolate between yellow (64, 130, 109) and green (0, 128, 0)
        r = int(np.interp(i, [0, num_points - 1], [64, 0]))
        g = int(np.interp(i, [0, num_points - 1], [130, 128]))
        b = int(np.interp(i, [0, num_points - 1], [109, 0]))
        colors.append(f'rgb({r}, {g}, {b})')  # Create HTML color string
    return colors

# Generate the color gradient

def convert_colors_to_rgb(colors):
    rgb_colors = []

    for color in colors:
        # Convert the color to RGB format
        rgb_color = tuple(int(255 * x) for x in color[:3])
        rgb_colors.append(f'rgb{rgb_color}')

    return rgb_colors


point_colors = convert_colors_to_rgb(point_colors1[1:-1])

# Create sliders for each point on both plots with modified CSS for handles
point_sliders1 = [widgets.FloatSlider(value=initial_points1[i], min=-10.0, max=10.0, step=0.01,
                                      description=f'Ωₚ(t = {i+1})', orientation='vertical',
                                      style={'handle_color': point_colors[i],
                                             'handle': 'background: linear-gradient(to bottom, yellow, green);'})  # Modify handle_color and handle
                  for i in range(9)]

point_sliders2 = [widgets.FloatSlider(value=initial_points2[i], min=-10.0, max=10.0, step=0.01,
                                      description=f'Ωₛ(t = {i+1})', orientation='vertical',
                                      style={'handle_color': point_colors[i],
                                             'handle': 'background: linear-gradient(to bottom, yellow, green);'})  # Modify handle_color and handle
                  for i in range(9)]

description_layout = widgets.Layout(width='700px', height='flex', align_items='flex-end')

laser1Label = widgets.HTML('<div style="text-align: justify; font-size: 16px;"> Each slider correspond to the amplitude of the laser at a specific time step, to define the shape of your first laser which moves the population from state 1 to state 2, adjust the respective sliders below.<div>', layout = description_layout)
laser2Label = widgets.HTML('<div style="text-align: justify; font-size: 16px;"> Each slider correspond to the amplitude of the laser at a specific time step, to define the shape of your second laser which moves the population from state 2 to state 3, adjust the respective sliders below.<div>', layout = description_layout)

# Create separate containers for the sliders in two columns
sliders_column1 = widgets.HBox(point_sliders1)
sliders_column2 = widgets.HBox(point_sliders2)

sliders_column11 = widgets.VBox([laser1Label, sliders_column1])
sliders_column22 = widgets.VBox([laser2Label, sliders_column2])

# Create Output widgets for the plots
plot_output1 = Output()
plot_output2 = Output()
plot_output3 = Output()

box_layout = widgets.Layout(align_items='stretch')

image_output = Output()

# Interactively update the plot based on slider values
interactive_plot = widgets.interactive(update_plot,
                    point1_1=point_sliders1[0],
                    point2_1=point_sliders1[1],
                    point3_1=point_sliders1[2],
                    point4_1=point_sliders1[3],
                    point5_1=point_sliders1[4],
                    point6_1=point_sliders1[5],
                    point7_1=point_sliders1[6],
                    point8_1=point_sliders1[7],
                    point9_1=point_sliders1[8],
                    point1_2=point_sliders2[0],
                    point2_2=point_sliders2[1],
                    point3_2=point_sliders2[2],
                    point4_2=point_sliders2[3],
                    point5_2=point_sliders2[4],
                    point6_2=point_sliders2[5],
                    point7_2=point_sliders2[6],
                    point8_2=point_sliders2[7],
                    point9_2=point_sliders2[8], 
                    point_colors1 = point_colors1)

# Create a button and its click event handler as before
stir_button = widgets.Button(description="Shoot the Lasers!", disabled=False)

currentScore = 0
highScore = 0

def run_all(ev):
    global highScore
    # Enable the submit button
    submit_button.disabled = False
    Give_me_the_dynamics(user_name)

# Connect the button's click event to the run_all function
stir_button.on_click(run_all)
stir_output = widgets.Output()


# Create some content to place inside the Accordion
content = widgets.HTML(value='<div style="text-align: justify; font-size: 16px;"> <b>Goal: Bring as much of the population to the third state using the two lasers. Find the most optimal laser pulse shape. \n After you have shaped your lasers, press \"Shoot the Lasers!\" button to see your result! After you are satisfied with your high score, press \"Submit to Scoreboard\" button and sending your score to the online Scoreboard by scanning the QR code!</b>', layout=description_layout)

laser_title1 = widgets.HTML(value="<h2>Laser One</h2>")
laser_title2 = widgets.HTML(value="<h2>Laser Two</h2>")

# Arrange the two slider columns side by side
sliders_layout = widgets.HBox([sliders_column11, sliders_column22])
sliders_layout.layout.align_items = 'center'


# Create a VBox to contain the interactive plot, the button, and the plot outputs
input = widgets.VBox([content, laser_title1, sliders_column11, plot_output1, laser_title2, sliders_column22, plot_output2])
input.layout.align_items = 'center'


def submit(ev):
    # when the submit button is clicked, we want to close this window and display the final page
    page2.close()
    with page2:
        clear_output(wait=True)
        display(final_page)

    if user_name == "":
        user_name_final = "Anonymous"
    else:
        user_name_final = user_name.replace(' ', '-')
    Stringbuilder = "https://gmscoreboard.com/api/set-score/?tagid=94b47096b96f2864e21376a548822b09&player=" + user_name_final + "&score=" + str(highScore)
    #print(Stringbuilder)
    qr = qrcode.make(Stringbuilder)

    # Save the qr code as an image
    qr.save('qr_code.png')

    with image_output:
        clear_output(wait=True)
        # Display the qr code
        image = Image('qr_code.png')
        display(image)


submit_button = widgets.Button(
                description='Submit to Scoreboard',
                disabled=True,
                button_style='success',
                tooltip='Click me',
                icon='check',
                layout=widgets.Layout(width='200px')
)

submit_button.on_click(submit)

right_layout = widgets.VBox([image1, plot_output3, submit_button])

right_layout.align = "bottom"

final = widgets.HBox([input, stir_button, right_layout])

final.layout.align_items = 'center'

# Wrapping all the widgets we have created into an Output widget which we can display and clear 
# as needed
page2 = widgets.Output()

#display(final)


# Adding a reset button, and a info widget that displays the infor from the previous page

# Create the button that will restart the game
restart_button = widgets.Button(
    description='Restart Game',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

# Increase the font size of the button text
restart_button.style.font_size = '16px'  # Adjust the font size as needed

# Increase the width of the button
restart_button.layout.width = '200px'  # Adjust the width as needed

# Create the title and description
thank_you_page = widgets.HTML(value="<h1>Thank You for Playing! To submit your score to the online scoreboard, please scan the QR code below.</h1>")

plug = widgets.HTML(value="<h1>To learn more about STIRAP, please check out our poster. To learn more about quantum computing and quantum games, please check out Quantum Game Club @ Purdue</h1>")

info_text = widgets.HTML(value='This program was developed by Hiram E. Diaz Berrios and Anderson Xu, under the guidance of Dr. Valentin Walther.')

plug2 = widgets.HTML(value="https://science.purdue.edu/walther/")

# image of quantum game club
plug3 = widgets.Image(value=open('Quantum_Game_Club.png', 'rb').read(),
    format='png',
    width=100,
    height=300,)


# Print out the QR code that is generated 
with image_output:
    clear_output(wait=True)
    # Display the qr code
    image = Image('qr_code.png')
    display(image)




# Create the VBox for the QR code and the resteart button
final_page = widgets.VBox([thank_you_page, plug, image_output, restart_button, info_text,plug2, plug3])

# Align the items in the VBox vertically in the center
final_page.layout.align_items = 'center'

page3 = widgets.Output()

display(main_box)

# Create a function that will clear the output and display the next page
def start_game(ev):
    global user_name
    user_name = name_input.value 
    main_box_output.close()
    with main_box_output:
        clear_output(wait=True)
        display(final)

# Connect the button's click event to the start_game function
start_button.on_click(start_game)



# Create a function that will clear the output and display the next page
def restart_game(ev):
    # reset all the sliders, high scores all variables used in the game to start a new session
    global highScore
    highScore = 0
    for slider in point_sliders1:
        slider.value = 0
    for slider in point_sliders2:
        slider.value = 0
    name_input.value = ''
    # disable the submit button
    submit_button.disabled = True
    # reset the state population plot to all zeros again
    with plot_output3:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.plot([0, 1], [0, 0], label='State 1', color = 'grey')
        ax.plot([0, 1], [0, 0], label='State 2', color = 'grey', linestyle='--')
        ax.plot([0, 1], [0, 0], label='Score', color = 'red')
        ax.set_ylim(-10, 10)
        ax.legend()
        ax.set_xlabel('time')
        ax.set_ylabel('Score')
        ax.set_title('State Populations [%]')
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['axes.labelsize'] = 18
        plt.rcParams['axes.titlesize'] = 20
        plt.tick_params(axis='both', which='major', labelsize=18)
        plt.show()


    page3.close()
    with page3:
        clear_output(wait=True)
    page2.close()
    with page2:
        clear_output(wait=True)
        display(main_box)

# Connect the button's click event to the restart_game function
restart_button.on_click(restart_game)

NameError: name 'widgets' is not defined